In [17]:
import pandas as pd
import sys, os

module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [18]:
from crimebb import *

In [19]:
from tqdm.notebook import tqdm

tqdm.pandas()

In [20]:
YEAR="2021"

In [21]:
DATA_PATH="../../data/"
CSV_PATH = f"{DATA_PATH}csv/{YEAR}/summary/"
CSV_PROCESSED = f"{DATA_PATH}csv/{YEAR}/processed/"

In [22]:
verifyDir(CSV_PROCESSED)

### Loading data

In [ ]:
chunk_size = 1000000

crime_reader = pd.read_csv(f"{CSV_PROCESSED}crimeBB_{YEAR}.csv", sep="\t", low_memory=False, iterator=True)
            
crimeBB_df = pd.DataFrame()

len_readed=chunk_size
while len_readed>=chunk_size:
    current_df = crime_reader.get_chunk(chunk_size).copy()
    current_df.drop_duplicates(inplace=True)

    crimeBB_df = pd.concat([crimeBB_df, current_df], ignore_index=True)

    len_readed = cur_posts_df.shape[0]

In [23]:
crimeBB_df = pd.read_csv(f"{CSV_PROCESSED}crimeBB_{YEAR}.csv", sep="\t", low_memory=False)

In [24]:
crimeBB_df

,post_id,site_id,board_id,thread_id,user_id,site_name,board_title,thread_title,username,content,user_reputation,post_data_creation
0,34,10,114,9,1174,forum.antichat.ru,Веб-уязвимости,ВЗЛОМ ФОРУМОВ,Algol,Адрес хоть дай...Первый раз слышу о таком ...,0,2002-05-31 06:36:00-03
1,637,10,114,113,1174,forum.antichat.ru,Веб-уязвимости,NaN,Algol,Кстати этот чат ломается ...,0,2002-08-05 16:15:00-03
2,710,10,114,137,1174,forum.antichat.ru,Веб-уязвимости,Клиенты для чата...,Algol,Чесно говоря это действительно большая редкост...,0,2002-08-12 08:08:00-03
3,491,10,114,99,1548,forum.antichat.ru,Веб-уязвимости,NaN,GPT cd-r74xl 24x,"Сорри, этт я неправильно указал ихний адрес. И...",0,2002-07-19 09:30:00-03
4,571,10,46,120,1968,forum.antichat.ru,Болталка,NaN,iRedX,Скоро будет обновление,9,2002-07-31 23:01:00-03
...,...,...,...,...,...,...,...,...,...,...,...,...
3365462,42116,20,49,8611,24965,torum6uvof666pzw.onion,General Discussions,Free stuff,tolerance2,What is that? I dont have javascript enabled s...,0,2019-08-06 21:00:00-03
3365463,42124,20,62,8454,22583,torum6uvof666pzw.onion,Marketplace Discussions,KEYLOGGER,mrkcuf,***LINK***lovelycloud[./memberlist.php?mode=vi...,0,2019-08-06 21:00:00-03
3365464,42129,20,62,8454,15407,torum6uvof666pzw.onion,Marketplace Discussions,KEYLOGGER,rIA,I wrote some keyloggers for android if you are...,0,2019-08-06 21:00:00-03
3365465,42134,20,62,8454,25064,torum6uvof666pzw.onion,Marketplace Discussions,KEYLOGGER,lovelycloud,ok..,0,2019-08-06 21:00:00-03


#### Detecting languages

In [13]:
language_to_eval = ["russian", "german", "english"]

In [14]:
if verifyFile(f"{CSV_PROCESSED}content_languages.csv"):
    content_df = pd.read_csv(f"{CSV_PROCESSED}content_languages.csv", sep="\t", low_memory=False)
    list_post = content_df[~content_df["language_detected"].isna()]["post_id"].values
else:
    content_df = crimeBB_df[["post_id", "site_id", "board_id", "thread_id", "content"]].copy()
    content_df = content_df[~content_df["content"].isna()].copy().drop_duplicates()
    # content_df["content_processed"] = content_df["content"].apply(lambda x: re.sub('[\\\\\'\"+@_!#$%^&*,;().<>?/\|\[\]}{~:=\n]', " ", x) )
    # content_df["content_processed"] = content_df["content_processed"].apply(str.lower)
    content_df["lang_ratio"] = np.nan
    content_df["language_detected"] = np.nan
    content_df["lang_correct_words"] = np.nan
    content_df["len_correct_words"] = np.nan
    content_df["lang_incorrect_words"] = np.nan
    content_df["len_incorrect_words"] = np.nan
    list_post = []

In [15]:
content_df

,post_id,site_id,board_id,thread_id,content,lang_ratio,language_detected,lang_correct_words,len_correct_words,lang_incorrect_words,len_incorrect_words
0,34,10,114,9,Адрес хоть дай...Первый раз слышу о таком ...,NaN,NaN,NaN,NaN,NaN,NaN
1,637,10,114,113,Кстати этот чат ломается ...,NaN,NaN,NaN,NaN,NaN,NaN
2,710,10,114,137,Чесно говоря это действительно большая редкост...,NaN,NaN,NaN,NaN,NaN,NaN
3,491,10,114,99,"Сорри, этт я неправильно указал ихний адрес. И...",NaN,NaN,NaN,NaN,NaN,NaN
4,571,10,46,120,Скоро будет обновление,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
3373784,42116,20,49,8611,What is that? I dont have javascript enabled s...,NaN,NaN,NaN,NaN,NaN,NaN
3373785,42124,20,62,8454,***LINK***lovelycloud[./memberlist.php?mode=vi...,NaN,NaN,NaN,NaN,NaN,NaN
3373786,42129,20,62,8454,I wrote some keyloggers for android if you are...,NaN,NaN,NaN,NaN,NaN,NaN
3373787,42134,20,62,8454,ok..,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
count = 1
for index, row in tqdm(content_df.iterrows()):
    
    lang_ratio, lang_detected, lang_badwords, lang_goodwords = detect_language_and_words(row["content"], language_to_eval=language_to_eval)
    
    content_df.loc[index, "lang_ratio"] = [lang_ratio]
    content_df.loc[index, "language_detected"] = lang_detected
    #content_df.loc[(d_manejo_key['TpRem'] == 'MANEJO'), "language_detected"] = lang_detected
    content_df.loc[index, "lang_correct_words"] = {"_":lang_goodwords}
    content_df.loc[index, "len_correct_words"] = len(lang_goodwords)
    content_df.loc[index, "lang_incorrect_words"] = {"_":lang_badwords}
    content_df.loc[index, "len_incorrect_words"] = len(lang_badwords)

    if count%10000==0: # grava cada 10 k muestras
        content_df.to_csv(f"{CSV_PROCESSED}content_languages.csv", sep='\t', index=False)
    
    count+=1
    

0it [00:00, ?it/s]

LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/russian.pickle[0m

  Searched in:
    - 'C:\\Users\\Adrian/nltk_data'
    - 'C:\\Users\\Adrian\\anaconda3\\envs\\crimebb\\nltk_data'
    - 'C:\\Users\\Adrian\\anaconda3\\envs\\crimebb\\share\\nltk_data'
    - 'C:\\Users\\Adrian\\anaconda3\\envs\\crimebb\\lib\\nltk_data'
    - 'C:\\Users\\Adrian\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - ''
**********************************************************************


In [ ]:
content_df.to_csv(f"{CSV_PROCESSED}content_languages.csv", sep='\t', index=False)

### Languages

In [ ]:
lang_post_df = content_df.pivot_table(columns=['language_detected'], aggfunc='size')
lang_post_df.sort_values(ascending=False, inplace=True)
lang_post_df

In [ ]:
mean_val = round(lang_post_df.mean(), 2)

import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(16,8))
ax.bar(lang_post_df.index.values, lang_post_df.values)

for x,y,p in zip(lang_post_df.index.values, lang_post_df.values, lang_post_df.values):
  plt.text(x, y, p)
plt.xlabel("Languages detected")
plt.ylabel("Number of documents")
plt.title(f"Number of documents per language, Avg: {mean_val}")
plt.show()

#### Russian

In [ ]:
# russian_df = content_df[content_df["language_detected"]=="russian"].copy()
# russian_df.info(), russian_df.shape, russian_df.iloc[0,:]

In [ ]:
# russian_content = ' '.join(russian_df["content"])

In [ ]:
# russian_dict = get_text_frequency(russian_content)
# len(russian_dict)

In [ ]:
# showWordCloud(russian_dict, f"Russian-Wordcloud", max_words=int(len(russian_dict)))

#### English

In [ ]:
# english_df = content_df[content_df["language_detected"]=="english"].copy()
# english_df.info(), english_df.shape, english_df.iloc[0,:]

In [ ]:
# english_content = ' '.join(english_df["content"])

In [ ]:
# english_dict = get_text_frequency(english_content)
# len(english_dict)

In [ ]:
# showWordCloud(english_dict, f"English-Wordcloud", max_words=int(len(english_dict)))

#### German

In [ ]:
# german_df = content_df[content_df["language_detected"]=="german"].copy()
# german_df.info(), german_df.shape, german_df.iloc[0,:]

In [ ]:
# german_content = ' '.join(german_df["content"])

In [ ]:
# german_dict = get_text_frequency(german_content)
# len(german_dict)

In [ ]:
# showWordCloud(german_dict, f"German-Wordcloud", max_words=int(len(german_dict)))